In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By

import time
import pandas as pd
import os

In [14]:
driver=webdriver.Chrome("chromedriver.exe")

driver.implicitly_wait(3)

C:\Users\LG\AppData\Local\Temp\ipykernel_27924\594694961.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome("chromedriver.exe")


1. 상품들 페이지 열기

In [15]:
site = "https://www.coupang.com/np/categories/420186"
driver.get(site)
time.sleep(1)

2. 상품 페이지의 리스트 부분을 FruitList에 넣기

In [16]:
FruitList = driver.find_element(By.ID, 'productList')

예시 코드

In [17]:
FruitList.find_element(By.XPATH, './li['+'1'+']').find_element(By.CLASS_NAME,'baby-product-link').get_attribute("href")

'https://www.coupang.com/vp/products/4650530853?itemId=5797807689&vendorItemId=73096293638&sourceType=CATEGORY&categoryId=420086'

./로 전체 XPATH를 입력하지 않고 FruitList의 XPATH에 이어서 붙인 XPATH를 사용할 수 있다

한 페이지 내의 주소들을 다 긁어오는 코드

In [18]:
i = 1
links = []
while True:
    try:
        temp = FruitList.find_element(By.XPATH, './li['+str(i)+']')
        link = temp.find_element(By.CLASS_NAME,'baby-product-link').get_attribute("href")
        links.append(link)
        i+=1
    except:
        break

In [19]:
# 총 저장된 링크 수
len(links)

60

In [20]:
# 마지막에 저장된 링크를 확인
links[-1]

'https://www.coupang.com/vp/products/1868163598?itemId=3175254926&vendorItemId=71162753297&sourceType=CATEGORY&categoryId=420086'

2. 저장한 상품 페이지 링크들을 하나씩 들어가면서 링크 가져오기

마지막에 저장된 링크를 예시로 진행

In [21]:
driver.get(links[-1])

리뷰 버튼 클릭

In [22]:
driver.find_element(By.NAME, 'sdp-review').click()
time.sleep(3)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[name="sbp-review"]"}
  (Session info: chrome=112.0.5615.138)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0045DCE3+50899]
	(No symbol) [0x003EE111]
	(No symbol) [0x002F5588]
	(No symbol) [0x003208F9]
	(No symbol) [0x00320AFB]
	(No symbol) [0x0034F902]
	(No symbol) [0x0033B944]
	(No symbol) [0x0034E01C]
	(No symbol) [0x0033B6F6]
	(No symbol) [0x00317708]
	(No symbol) [0x0031886D]
	GetHandleVerifier [0x006C3EAE+2566302]
	GetHandleVerifier [0x006F92B1+2784417]
	GetHandleVerifier [0x006F327C+2759788]
	GetHandleVerifier [0x004F5740+672048]
	(No symbol) [0x003F8872]
	(No symbol) [0x003F41C8]
	(No symbol) [0x003F42AB]
	(No symbol) [0x003E71B7]
	BaseThreadInitThunk [0x767B7D49+25]
	RtlInitializeExceptionChain [0x7765B74B+107]
	RtlClearBits [0x7765B6CF+191]


In [ ]:
review = driver.find_element(By.CLASS_NAME, "js_reviewArticleListContainer")

In [ ]:
i=0 # i는 현재까지 저장된 리뷰의 수를 의미
j=0 # j는 보여지는 리뷰들의 순서와 관련
data = []
while True:
    i += 1
    j += 1
    try:    # 리뷰 평점 저장
        data.append(review.find_element(By.XPATH, "./article["+str(j)+"]/div[1]/div[3]/div[1]/div").get_attribute("data-rating"))
    except: # 보여지는 리뷰들을 다 불러와서 그 이후의 리뷰가 없어 에러가 나면 실행 - 다음 리뷰들을 띄운다
        j = 0
        page = driver.find_element(By.XPATH, "/html/body/div[2]/section/div[2]/div[7]/ul[2]/li[3]/div/div[6]/section[4]/div[3]")
        p = int(page.get_attribute("data-page"))%10+2
        page.find_element(By.XPATH, "./button["+str(p)+"]").click()
        time.sleep(3)
    if i >= 10:
        break

이제 저장된 사이트 링크 모두를 대상으로 실행해보기

In [ ]:
for i in links:
    driver.get(i)
    time.sleep(2)   # 페이지 이동 시에 완전히 로딩되지 않았음에도 되는 경우 방지용 3초 대기
    
    title = driver.find_element(By.CLASS_NAME, "prod-buy-header__title").text # 상품명 저장
    
    driver.find_element(By.NAME, 'review').click()
    time.sleep(2)
    review = driver.find_element(By.CLASS_NAME, "js_reviewArticleListContainer")

    i=0 # i는 현재까지 저장된 리뷰의 수를 의미
    j=0 # j는 보여지는 리뷰들의 순서와 관련
    score = []

    while True:
        i += 1
        j += 1
        try:    # 리뷰 평점 저장
            score.append(review.find_element(By.XPATH, "./article["+str(j)+"]/div[1]/div[3]/div[1]/div").get_attribute("data-rating"))
        except: # 보여지는 리뷰들을 다 불러와서 그 이후의 리뷰가 없어 에러가 나면 실행 - 다음 리뷰들을 띄운다
            j = 0
            try:
                page = driver.find_element(By.XPATH, "/html/body/div[2]/section/div[2]/div[7]/ul[2]/li[3]/div/div[6]/section[4]/div[3]")
                p = int(page.get_attribute("data-page"))%10+2
                page.find_element(By.XPATH, "./button["+str(p)+"]").click()
                time.sleep(1)
            except:
                break        
        if i >= 10:
            break
        
    dic = {"review-score": score}
    df = pd.DataFrame(dic)
    df.to_csv(os.getcwd()+"\\"+title+'.csv', index=False, encoding='utf-8-sig')
    print(title+" 저장 완료")
    

곰곰 국내산 딸기 저장 완료
돌 바나나 저장 완료
성주벽진농협 당도선별 미니참외 저장 완료
산지직송 수토마켓 당도선별 수박 저장 완료
곰곰 국내산 블루베리(대) 저장 완료
스윗토 스테비아 대추방울토마토 저장 완료
프레쉬팜 골드 파인애플 스틱 저장 완료
스위티오 Dole 바나나 저장 완료
퓨어스펙 고당도 오렌지 저장 완료
만나 프레시 믹스 파인 포도 방울 150g x 3p + 사과 포도 방울 150g x 3p 저장 완료
곰곰 당도선별 수박 (함안) 저장 완료
델몬트 골드파인 저장 완료
산지직송 푸드베이스 산딸기 저장 완료
스테비아 스윗토 대추방울토마토 저장 완료
충남세도 GAP 인증 대추방울토마토 저장 완료
제스프리 썬골드 키위 저장 완료
썬키스트 고당도 오렌지 중대과 저장 완료
Dole 스위티오 바나나 저장 완료
곰곰 당도선별 성주참외 저장 완료
제스프리 왕점보 썬골드키위 저장 완료
새벽경매 시원달콤한 수박 저장 완료
제스프리 썬골드 키위 저장 완료
곰곰 농협 영주 당도선별 세척사과 저장 완료
프레샤인 GAP 인증 충주 못난이사과 저장 완료
곰곰 안동 시나노 골드사과 저장 완료
제스프리 썬골드 키위 저장 완료
달콤한농부 스테비아 스타마토 저장 완료
수플린 태국산 망고스틴 5~6입 저장 완료
델몬트 허니글로우 파인스틱 6입 저장 완료
퓨어스펙 고당도 오렌지 중대과 저장 완료
파인토 스테비아 방울토마토 저장 완료
농협 당도선별 수박 저장 완료
만나다 미국 고당도 오렌지 저장 완료
GAP 완숙토마토 저장 완료
성주월항농협 당도선별 참외 저장 완료
곰곰 GAP 당도선별 제주 카라향 저장 완료
산지직송 수토마켓 당도선별 수박 저장 완료
곰곰 달콤한 스테비아 대추방울토마토 저장 완료
딜리조이 시나노 골드 못난이 사과 저장 완료
호재준 유기가공식품 인증 블루베리 (냉동) 저장 완료
가족 고당도 시나노골드 사과 저장 완료
제스프리 점보 썬골드키위 저장 완료
한반도 당도선별 수박 저장 완료
성주벽진농협 당도선별 미니참외 저장 완료
GAP 당도선별 성주참외 저장 완료
엠팍 고당

In [ ]:
driver.quit()

참고하면 좋은 사이트

https://www.selenium.dev/selenium/docs/api/java/org/openqa/selenium/WebElement.html